In [ ]:
# cell 1: Install dependencies
!pip install -q langchain langchain-community langchain-huggingface faiss-cpu sentence-transformers bitsandbytes accelerate
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q transformers datasets